In [1]:
import random, numpy as np, torch
def seed_everything(seed=11): 
    random.seed(seed) 
    np.random.seed(seed) 
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed) 
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
seed_everything()

In [2]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427

import unicodedata, re 
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

def readLangs(lang1, lang2):
    print("Reading lines...")

    # Read the file and split into lines
    with open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8') as fr:
        lines = fr.readlines() 
        print('First 3 lines in the file:', "', '".join([i.strip().__repr__() for i in lines[:3]]), '\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
  
    pairs = [list(reversed(p)) for p in pairs]
    input_lang  = Lang(lang2)
    output_lang = Lang(lang1)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print(f"Read {len(pairs)} sentence pairs")
    print('First 3 sentence pairs:', pairs[:3], '\n')
    pairs = [pair for pair in pairs 
            if len(pair[0].split(' ')) < MAX_LENGTH and \
               len(pair[1].split(' ')) < MAX_LENGTH and \
                   pair[1].startswith(eng_prefixes)]
    print("Trimmed to %s sentence pairs" % len(pairs)) 
    print('First 3 trimmed sentence pairs:', pairs[:3])
    print('Random sentence pair', random.choice(pairs), '\n')
    for pair in pairs:
        input_lang .addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:", input_lang .name, input_lang .n_words, output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra')

Reading lines...
First 3 lines in the file: 'Go.\tVa !'', ''Run!\tCours\u202f!'', ''Run!\tCourez\u202f!' 

Read 135842 sentence pairs
First 3 sentence pairs: [['va !', 'go'], ['cours !', 'run !'], ['courez !', 'run !']] 

Trimmed to 11445 sentence pairs
First 3 trimmed sentence pairs: [['je vais bien', 'i m ok'], ['ca va', 'i m ok'], ['je suis gras', 'i m fat']]
Random sentence pair ['je crains qu il ne soit trop tard', 'i am afraid it s too late'] 

Counted words: fra 4601 eng 2991


In [3]:
import torch.nn as nn
import torch.nn.functional as F 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Compute Resource:', device.type, '\n')

SOS_token = 0
EOS_token = 1

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output = F.relu(self.embedding(decoder_input)) 
            decoder_output, decoder_hidden = self.gru(decoder_output, decoder_hidden) 
            decoder_outputs.append(self.out(decoder_output)) 
            if target_tensor is not None: 
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing: Feed the target as the next input
            else: 
                _, topi = decoder_output.topk(1)           # Without teacher forcing: use its own predictions as the next input
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, None, None

hidden_size = 128
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

Compute Resource: cuda 



In [4]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def get_dataloader(batch_size):
    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader 

batch_size = 32
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

In [5]:
from torch import optim

n_epochs = 80
learning_rate=0.001
print_every=5 
print_loss_total = 0  # Reset every print_every 

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

for epoch in range(1, n_epochs + 1):  
    total_loss = 0
    for data in train_dataloader:
        input_tensor, target_tensor = data 
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()
    print_loss_total += total_loss / len(train_dataloader)
    
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print(f'{epoch}, {epoch / n_epochs * 100}%, {print_loss_avg:.4f}')

torch.save(decoder.state_dict(), 'seq2seqDecoder.pth')
torch.save(encoder.state_dict(), 'seq2seqEncoder.pth')

5, 6.25%, 1.6905
10, 12.5%, 0.9279
15, 18.75%, 0.6156
20, 25.0%, 0.4277
25, 31.25%, 0.3078
30, 37.5%, 0.2286
35, 43.75%, 0.1743
40, 50.0%, 0.1349
45, 56.25%, 0.1082
50, 62.5%, 0.0898
55, 68.75%, 0.0746
60, 75.0%, 0.0648
65, 81.25%, 0.0566
70, 87.5%, 0.0521
75, 93.75%, 0.0467
80, 100.0%, 0.0434


In [6]:
#decoder.load_state_dict(torch.load('seq2seqDecoder.pth'))
#encoder.load_state_dict(torch.load('seq2seqEncoder.pth'))
encoder.eval()
decoder.eval()
for i in range(10):
    pair = random.choice(pairs)
    print('Src:', pair[0])
    print('Tgt:', pair[1])
    sentence = pair[0] 
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    output_words = decoded_words 
    output_sentence = ' '.join(output_words)
    print('Pred:', output_sentence)
    print('')

Src: j ai plus d experience que tom
Tgt: i m more experienced than tom
Pred: i years always to for tom been tom <EOS>

Src: je suis a court d energie
Tgt: i m running out of energy
Pred: i s to <EOS>

Src: il est candidat au congres
Tgt: he s running for congress
Pred: he he about s for this <EOS>

Src: je suis tienne et tu es mien
Tgt: i am yours and you are mine
Pred: i am about you every <EOS>

Src: elle est desireuse de visiter l europe
Tgt: she is anxious to visit europe
Pred: she is out because is <EOS>

Src: j ai toujours soif
Tgt: i m still thirsty
Pred: i my my still <EOS>

Src: je creve de froid
Tgt: i m freezing cold
Pred: i re of is <EOS>

Src: je ne suis pas assez bon pour vous
Tgt: i m not good enough for you
Pred: i aren not not a and <EOS>

Src: tu n as pas totalement tort
Tgt: you re not entirely wrong
Pred: you you not not wrong <EOS>

Src: c est un expert du lancer de couteaux
Tgt: he s an expert at throwing knives
Pred: he re the <EOS>

